In [17]:
import sys 
import csv
sys.path.append("..")
from src.utils.constantes import DATA_CLEAN_PATH
from src.utils.funciones import porcentaje


In [18]:
def separar_por_trimestre(dict_anios): 
    dict_final = {}
    for anio in dict_anios:
        dict_temporal = {}
        
        for filas in dict_anios[anio]:
            trimestre = filas[2]
            if trimestre not in dict_temporal: 
                dict_temporal[trimestre] = []
            dict_temporal[trimestre].append(filas)

        dict_final[anio] = dict_temporal
    return dict_final


def agrupar_por_anio_y_trimestre(filas, col_anio=1):
    grupos = {}
    for fila in filas:
        anio = fila[col_anio]
        if anio not in grupos:
            grupos[anio] = []
        grupos[anio].append(fila)
    grupos_final = separar_por_trimestre(grupos)
    return grupos_final

def agrupar_por_aglomerado (new_list_i): 
    viejo_dict_aglomerados = {}
    for elem in new_list_i:
        if elem[7] not in viejo_dict_aglomerados: 
            viejo_dict_aglomerados[elem[7]] = []
        viejo_dict_aglomerados[elem[7]].append(elem)
    return viejo_dict_aglomerados

def obtener_ultimo_archivo (dict_trimestres): 
    max_anio = max(dict_trimestres.keys())
    max_trimestre = max(dict_trimestres[max_anio].keys())
    return dict_trimestres[max_anio][max_trimestre]
    


Abriendo archivo individuos

In [19]:
archivo_clean_path = DATA_CLEAN_PATH / "usu_clean_individual.csv"

archivo_individuos = archivo_clean_path.open("r",encoding="utf-8") 
reader = csv.reader(archivo_individuos,delimiter=";")
header = next(reader)

new_list_i = []

for elem in reader:
    new_elem = elem[0].split(',')
    new_list_i.append(new_elem)

aglomerados_individuos = agrupar_por_aglomerado(new_list_i)
anios_trimestres_individuos = agrupar_por_anio_y_trimestre(new_list_i)
ultimos_individuos = obtener_ultimo_archivo(anios_trimestres_individuos)


Abriendo archivo hogares


In [20]:
archivo_clean_path = DATA_CLEAN_PATH / "usu_clean_hogar.csv"

archivo_hogares = archivo_clean_path.open("r",encoding="utf-8") 
reader = csv.reader(archivo_hogares,delimiter=";")
header = next(reader)

new_list_h = []

for elem in reader:
    new_elem = elem[0].split(',')
    new_list_h.append(new_elem)

aglomerados_hogares = agrupar_por_aglomerado(new_list_h)
años_trimestres_hogares = agrupar_por_anio_y_trimestre(new_list_h)
ultimos_hogares = obtener_ultimo_archivo(años_trimestres_hogares)


1- A partir de la información de cada año contenida en el dataset se debe informar, año tras año, el porcentaje de personas mayores a 6 años capaces e incapaces de leer y escribir.
Importante: tomar la información del último trimestre de cada año.

### Tabla individual  
1- ANO4 N (4) Año de relevamiento (4 dígitos)

2- TRIMESTRE N (1) Ventana de observación  
- = 1° trimestre  
- = 2° trimestre  
- = 3° trimestre  
- = 4° trimestre  

9- PONDERA N (6) Ponderación  

12- CH05 date Fecha de nacimiento
(día, mes y año) --> ver como esta dado el dato

13- CH06 N (2) ¿Cuántos años cumplidos tiene?

16- CH09 N (1) ¿Sabe leer y escribir?
1 = Sí
2 = No
3 = Menor de 2 años

________________________________________________________
Columnas usadas: 2 - 1 - 9 - 13 - 16


In [ ]:
def contar_alfabetismo(filas):
    """
    Recorre las filas y calcula los totales ponderados de:
    alfabetos, analfabetos, casos desconocidos y población mayor a 6 años.
    """
    
    alfabetos = analfabetos = desconocido = poblacion = 0

    for elem in filas:
        pondera = int(elem[9]) # es elem[8] no 9, pq es lista y empiezan en 0
        edad = int(elem[13])
        if edad >= 6:
            poblacion += pondera
            if elem[16] == "1":
                alfabetos += pondera
            elif elem[16] == "2":
                analfabetos += pondera
            else:
                desconocido += pondera
    return alfabetos, analfabetos, desconocido, poblacion

def calcular_porcentajes(alfabetos, analfabetos, desconocido, poblacion):
    """
    Calcula los porcentajes de alfabetos, analfabetos y desconocidos sobre la población total.
    """
    return (
        porcentaje(alfabetos, poblacion),
        porcentaje(analfabetos, poblacion),
        porcentaje(desconocido, poblacion),
    )

def obtener_ultimo_trimestre(dict_anio_trimestre,anio):
    return max(dict_anio_trimestre[anio].keys())


def analizar_analfabetismo(dict_anio_trimestre):
    """
    Para cada año, calcula e imprime el último trimestre disponible y los
    porcentajes de alfabetos, analfabetos y casos desconocidos en población > 6 años.
    """
    for anio in dict_anio_trimestre:
        trimestre = obtener_ultimo_trimestre(dict_anio_trimestre, anio)
        filas = dict_anio_trimestre[anio][trimestre]
        alfabetos, analfabetos, desconocido, poblacion = contar_alfabetismo(filas)
        por_alfa, por_ana, por_desc = calcular_porcentajes(alfabetos, analfabetos, desconocido, poblacion)

        print(f"Año {anio}, Trimestre {trimestre}")
        print(f"  - Alfabetismo: {por_alfa:.2f}%")
        print(f"  - Analfabetismo: {por_ana:.2f}%")
        print(f"  - Desconocido: {por_desc:.2f}%\n")


def actividad_1(individuos):
    analizar_analfabetismo(individuos)



In [22]:
actividad_1(anios_trimestres_individuos)

Año 2024, Trimestre 2
  - Alfabetismo: 98.57%
  - Analfabetismo: 1.39%
  - Desconocido: 0.04%

Año 2023, Trimestre 4
  - Alfabetismo: 98.47%
  - Analfabetismo: 1.46%
  - Desconocido: 0.07%



2. A partir de un año y trimestre elegido por el usuario informar el porcentaje de personas no nacidas en Argentina que hayan cursado un nivel universitario o superior.

In [41]:
def es_extranjero(valor_ch15):
    return valor_ch15 in ['4', '5']

def es_universitario(valor_ch12):
    return valor_ch12 in ['7', '8']

def contar_extranjeros_universitarios(filas):
    # Calcula la cantidad de extranjeros, universitarios y ambos combinados
    total_extranjeros = total_universitarios = total_universitarios_extranjeros = 0

    for elem in filas:
        pondera = int(elem[9])
        ch12 = elem[19]  # nivel educativo
        ch15 = elem[22]  # lugar de nacimiento

        if es_extranjero(ch15):
            total_extranjeros += pondera
            if es_universitario(ch12):
                total_universitarios_extranjeros += pondera

        if es_universitario(ch12):
            total_universitarios += pondera

    return total_extranjeros, total_universitarios, total_universitarios_extranjeros

def porcentajes_universitarios_extranjeros(total_extranjeros, total_universitarios, total_universitarios_extranjeros):
    """
    Calcula tres porcentajes:
    - % de universitarios extranjeros sobre el total de extranjeros
    - % de universitarios extranjeros sobre el total de universitarios
    - % de universitarios extranjeros sobre el total de la muestra combinada
    """
    total_muestra = total_extranjeros + total_universitarios  # para referencia general

    por_sobre_extranjeros = porcentaje(total_universitarios_extranjeros, total_extranjeros) if total_extranjeros else 0

    por_sobre_universitarios = porcentaje(total_universitarios_extranjeros, total_universitarios) if total_universitarios else 0

    por_sobre_muestra = porcentaje(total_universitarios_extranjeros, total_muestra) if total_muestra else 0

    return round(por_sobre_extranjeros, 2), round(por_sobre_universitarios, 2), round(por_sobre_muestra, 2)

def analizar_universitarios_extranjeros(anios_trimestres_individuos):
    """
    Para cada año, analiza el último trimestre disponible y muestra:
    - % de universitarios extranjeros sobre extranjeros
    - % de universitarios extranjeros sobre universitarios
    - % sobre el total combinado
    """
    
    anio = input("ingrese el año a buscar")    
    trimestre = input(f"ingrese el trimestre a buscar de {anio}")
    filas = anios_trimestres_individuos[anio][trimestre]

    total_ext, total_univ, total_ext_univ = contar_extranjeros_universitarios(filas)
    p1, p2, p3 = porcentajes_universitarios_extranjeros(total_ext, total_univ, total_ext_univ)

    print(f"Año {anio}, Trimestre {trimestre}")
    print(f"  - % Univ. Extranjeros / Total Extranjeros: {p1:.2f}%")
    print(f"  - % Univ. Extranjeros / Total Univ.: {p2:.2f}%")
    print(f"  - % Univ. Extranjeros / Muestra combinada: {p3:.2f}%\n")

In [42]:
analizar_universitarios_extranjeros(anios_trimestres_individuos)

Año 2024, Trimestre 2
  - % Univ. Extranjeros / Total Extranjeros: 20.96%
  - % Univ. Extranjeros / Total Univ.: 5.70%
  - % Univ. Extranjeros / Muestra combinada: 4.48%



3. A partir de la información contenida en el dataset informar el año y trimestre donde hubo menor desocupación.

Criterio usado:  
Poblacion en condiciones de trabajar: aproximadamente entre 18 y 65 años varia dependiendo del sexo.  
Población Activa: incluye población ocupada y desempleados (desocupados, personas que no tienen trabajo aunque lo buscan activamente).  
Población Inactiva: gente que no tiene trabajo pero tampoco lo busca.  

Para el ejercicio se cosiderara:  
Población activa = Ocupados + Desocupados  
Tasa de paro o desempleo = (Población desocupada / Población activa) x 100

In [48]:
def calcular_tasa_desocupacion(filas):
    """
    Calcula el total de desocupados y total activos (ocupados + desocupados) ponderados.
    """
    total_desocupados = total_activos = 0

    for fila in filas:
        estado = fila[27]
        pondera = int(fila[8])

        if estado == '2':  # Desocupado
            total_desocupados += pondera
            total_activos += pondera
        elif estado == '1':  # Ocupado
            total_activos += pondera

    return porcentaje(total_desocupados, total_activos)

def buscar_menor_desocupacion(anios_trimestres_individuos):
    """
    Recorre todos los años y trimestres y encuentra el de menor tasa de desocupación.
    """

    menor_tasa = None
    anio_menor = None
    trimestre_menor = None
    
    for anio in anios_trimestres_individuos:

        for trimestre in anios_trimestres_individuos[anio]:

            filas = anios_trimestres_individuos[anio][trimestre]
            tasa = calcular_tasa_desocupacion(filas)

            if (menor_tasa is None) or (tasa < menor_tasa):
                menor_tasa = tasa
                anio_menor = anio
                trimestre_menor = trimestre

    print(f"\nEl menor porcentaje de desocupación fue de {menor_tasa:.2f}%")
    print(f"Año: {anio_menor}, Trimestre: {trimestre_menor}")

In [49]:
buscar_menor_desocupacion(anios_trimestres_individuos)


El menor porcentaje de desocupación fue de 4.71%
Año: 2023, Trimestre: 4


Abriendo el archivo Hogares



4.  Ranking de los 5 aglomerados con mayor porcentaje de hogares con dos o más 
ocupantes con estudios universitarios o superiores finalizados. Información obtenida 
a partir del par de archivos más recientes. 

In [28]:
def tiene_dos_ocupantes (elem): 
    return int(elem[64]) >= 2 
def es_universitario (elem): 
    return elem[26] in ["5","6"]

def activ4 (dict_trimestres_i, dict_trimestres_h):
# busca el ultimo archivo de Hogares y Individuos 

    ## busca el año mas reciente de cada uno de los diccionarios (divididos por triemstre)
    max_anio_i = max(dict_trimestres_i.keys())

    max_anio_h = max(dict_trimestres_h.keys())

    ## busca el trimestre mas reciente para cada uno de los diccionarios (en el año maximo obtenido anteriormmnte)
    max_tri_i = max(dict_trimestres_i[max_anio_i].keys())

    max_tri_h = max(dict_trimestres_h[max_anio_h].keys())

    ## guarda los hogares e individuos encuestados del periodo mas reciente (trimestre y año tomados anteriormente)
    ultimos_hogares = dict_trimestres_h[max_anio_h][max_tri_h]

    ultimos_individuos  = dict_trimestres_i[max_anio_i][max_tri_i]
    
    ## declarando listas y dicts
    hogares_CODUSU, individuos_CODUSU, hogares, individuos = [], [], [], [] 

    poblacion_total_aglomerado, encuestas_finales, porcentajes_finales = {}, {}, {}

# busca dentro de los hogares e individuos mas recientes si cumplen con las condiciones

    ## guarda los hogares que cumplen con la condicion y su respectivo CODUSU 
    for elem in ultimos_hogares: 
        if not tiene_dos_ocupantes(elem):
            continue
        hogares.append(elem)
        hogares_CODUSU.append(elem[0]) 

    ## guarda los individuos que cumplen con la condicion y su respectivo CODUSU 
    for elem in ultimos_individuos:
        if not es_universitario(elem): 
            continue 
        individuos.append(elem)
        individuos_CODUSU.append(elem[0])

# busca dentro de los hogares e intenta encontrar los que cumplen las condiciones de individuos y hogares

    for elem in hogares:
    
    ## añade lo que pondera cada aglomerado en un aglomerado_pondera
        if elem[7] not in poblacion_total_aglomerado:
            poblacion_total_aglomerado[elem[7]] = 0

        poblacion_total_aglomerado[elem[7]] += int(elem[8])

    ## si no cumple la condicion de individuos se ignora ya cumple con la condicion de hogares, por ende si cumple esta cumple con ambas
        if elem[0] not in individuos_CODUSU: 
            continue 

    ## añade las encuestas finales( que cumplen con ambas condiciones) a encuestas_finales, separado por aglomerado 
        if elem[7] not in encuestas_finales:
            encuestas_finales[elem[7]] = []

        encuestas_finales[elem[7]].append(elem)

    for aglomerado in encuestas_finales:
    ## crea un diccionario dividido por aglomerado con lo que ponderan las encuestas finales 
        poblacion_final_aglomerado = 0

        if aglomerado not in porcentajes_finales:
            porcentajes_finales[aglomerado] = 0 

        for elem in encuestas_finales[aglomerado]: 
            poblacion_final_aglomerado += int(elem[8])
        
    ## porcentaje de hogares que cumplen con las condiciones, dividido por aglomerado 
        porcentajes_finales[aglomerado] = porcentaje(poblacion_final_aglomerado, poblacion_total_aglomerado[aglomerado])
    

    ## encuentra el aglomerado con el mayor porcentaje 
    max_porcentaje = 0 
    max_aglomerado = 0 
    for aglomerado in porcentajes_finales: 
        if porcentajes_finales[aglomerado] > max_porcentaje:
            max_aglomerado = aglomerado
            max_porcentaje = porcentajes_finales[aglomerado]
        
    print(f'el aglomerado con mayor porcentaje fue el: {max_aglomerado} con un porcentaje de {max_porcentaje:.2f}')



In [29]:
activ4(anios_trimestres_individuos,años_trimestres_hogares)

el aglomerado con mayor porcentaje fue el: 32 con un porcentaje de 72.42


- 5. Informar para cada aglomerado el porcentaje de viviendas ocupadas por sus
propietarios.

In [57]:
def porcentaje_propietarios_por_aglomerado(dict_aglomerados, col_aglomerado=7, col_pondera=8, col_tenencia=37):
    
    resultados = {}
    
    for aglomerado in dict_aglomerados:
        for elem in dict_aglomerados[aglomerado]:
            try:
                pondera = int(elem[col_pondera])
                tenencia = elem[col_tenencia]

                if aglomerado not in resultados:
                    resultados[aglomerado] = {'total': 0, 'propietarios': 0}
        
                if tenencia in ('1', '2'):  # Propietario total o parcial
                    resultados[aglomerado]['propietarios'] += pondera
            
                resultados[aglomerado]['total'] += pondera 
            except:
                continue
    
    # Calcular porcentaje para cada aglomerado
    porcentaje_max = 0
    for aglomerado, valores in sorted(resultados.items()):
        
        porcentaje_prop = porcentaje(valores["propietarios"], valores["total"])
        
        if porcentaje_prop > porcentaje_max:
            porcentaje_max = porcentaje_prop
            aglomerado_max = aglomerado 
    
    print(f"el Aglomerado {aglomerado_max} tuvo el mayor porcentaje, con: {porcentaje_max:.2f}% viviendas de propietarios")



In [58]:
porcentaje_propietarios_por_aglomerado(aglomerados_hogares)

el Aglomerado 18 tuvo el mayor porcentaje, con: 85.90% viviendas de propietarios


6. Informar el aglomerado con mayor cantidad de viviendas con más de dos ocupantes
y sin baño. Informar también la cantidad de ellas.

In [32]:
archivo_hogares.close()
archivo_individuos.close()